In [0]:
#필요한 패키지를 가져오기(numpy,warnings,sklearn등등)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import log_loss

In [0]:
#xgboost
from xgboost import plot_importance
from xgboost import XGBClassifier

In [0]:
!ls
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)


sample_data
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

drive  gdrive  sample_data


In [0]:
#데이터셋 불러오기
train=pd.read_csv('/content/gdrive/My Drive/train.csv')
test=pd.read_csv('/content/gdrive/My Drive/test.csv')
X_test=test
test.head()
train.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [0]:
#type을 숫자로 labeling
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(train.type)
type_encoded=le.transform(train.type)
train.type=type_encoded
train.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,1,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,1,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,1,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,1,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,16,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [0]:

y = train['type']
X = train.drop(columns=['type'], axis=1)
predict_test=test

#train을 또 training set과 test set으로 나누기
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size= 0.3,random_state=21)


In [0]:
#XGB 모델 만들기
xgb=XGBClassifier(n_estimators=2000,scoring='log_loss',
                        n_jobs=4,
                        max_depth=15,
                        learning_rate=0.05,
                        gamma = 0.02,
                        subsample = 0.9,
                        colsample_bytree=0.9,
                        tree_method='gpu_hist')


In [0]:
xgb.fit(X,y)
print('training set accuracy:',xgb.score(X_train,y_train))
print(xgb.predict(predict_test))

training set accuracy: 1.0
[16  1  0 ...  1  1 10]


In [0]:
predicitons=xgb.predict_proba(predict_test)
print(y.shape)
print(predictions.shape)

(199991,)
(10009,)


In [0]:

#log loss값을 구하기
predictions=xgb.predict_proba(X_test)
score=log_loss(y_true=y_test,y_pred=predictions)


In [0]:
print(score)



0.0038261253987284254


In [0]:
gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=250,learning_rate=0.5,random_state=21)
gbrt.fit(x_train,y_train)
print('training set accuracy:',gbrt.score(x_train,y_train))
print(gbrt.predict(x_test))

training set accuracy: 0.8081450439118018
[15.40295951  2.90110171  4.21327486 ...  0.79216795  1.16372927
  9.94186616]


In [0]:
loss=log_loss()

TypeError: ignored

In [0]:
df=pd.DataFrame(gbrt.predict(test_x))

In [0]:
df

,0
0,15.402960
1,2.901102
2,4.213275
3,3.847985
4,15.929720
...,...
10004,4.406664
10005,6.108065
10006,0.792168
10007,1.163729


In [0]:
forest=RandomForestClassifier(n_estimators=21,n_jobs=-1)
forest.fit(x_train,y_train)
print('training set accuracy:',forest.score(x_train,y_train))
print(forest.predict(test_x))

training set accuracy: 0.9980499122460511
[16  1  0 ...  1  1 10]


In [0]:
df2=pd.DataFrame(forest.predict(test_x))

In [0]:
df2

,0
0,16
1,1
2,0
3,4
4,16
...,...
10004,4
10005,4
10006,1
10007,1


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.02,
              learning_rate=0.05, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=2000, n_jobs=4,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [0]:
xgb.predict(test_x)

array([16,  1,  0, ...,  1,  1, 10])

In [0]:

from sklearn.metrics import log_loss
y_pred_rnd_prob = rnd_model.predict_proba(x_test)
log_loss(y_true=x_test, y_pred=y_pred_rnd_prob)

NameError: ignored